This notebook will contain code relevant to the logistic regression model.  

# Load files and Import packages

In [1]:
## Math
from math import floor
from statistics import *

## Handling Arrays and Dataframes
import pandas as pd
import numpy as np

## Train Test Set Splitting
from sklearn.model_selection import train_test_split

## Sampling
from imblearn.over_sampling import SMOTENC
from sklearn.utils import resample

## Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Model Train and Testing
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as met
import matplotlib.pyplot as plt

## PCA
from sklearn.decomposition import PCA

## Fine-tuning
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

## LIME
!pip install lime
import lime
import lime.lime_tabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=3571891b2be93722ab91e577e1017b391cac5f5fb85144f222716f7c19d123d2
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [2]:
## mount drive and retrieve labelled data

from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/MyDrive/DSA4266_Tundra")

m_data_raw = pd.read_csv("merged_data.csv")
#m_data_raw = m_data_raw.iloc[:,1:]
m_data_raw.reset_index(drop = True, inplace = True)

Mounted at /content/drive


In [3]:
m_data_raw.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


# Data Preprocessing

## Remove correlated variables

In [4]:
corr = m_data_raw.corr()
corr.style.background_gradient(cmap='coolwarm')

<ipython-input-4-9d2b1d4d835e>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = m_data_raw.corr()


,transcript_position,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,label
transcript_position,1.000000,-0.010329,-0.047637,-0.079866,-0.020504,-0.045754,-0.068146,-0.006597,-0.029134,0.025372,0.018985
-1_flank_length,-0.010329,1.000000,0.127075,0.094277,0.001115,0.009783,-0.029552,0.014739,-0.018274,-0.012424,0.009458
-1_flank_std,-0.047637,0.127075,1.000000,0.380163,0.005668,0.059138,-0.030368,0.012969,0.015944,-0.024378,0.024992
-1_flank_mean,-0.079866,0.094277,0.380163,1.000000,0.069187,0.057715,0.239609,0.016304,0.121022,0.008591,0.099535
central_length,-0.020504,0.001115,0.005668,0.069187,1.000000,0.128894,0.150908,0.035523,0.055304,-0.055893,0.006456
central_std,-0.045754,0.009783,0.059138,0.057715,0.128894,1.000000,0.438944,0.028204,0.147590,-0.429502,-0.000377
central_mean,-0.068146,-0.029552,-0.030368,0.239609,0.150908,0.438944,1.000000,0.051361,0.392869,-0.247272,0.073311
+1_flank_length,-0.006597,0.014739,0.012969,0.016304,0.035523,0.028204,0.051361,1.000000,0.095774,-0.003797,0.011718
+1_flank_std,-0.029134,-0.018274,0.015944,0.121022,0.055304,0.147590,0.392869,0.095774,1.000000,-0.078569,0.007753
+1_flank_mean,0.025372,-0.012424,-0.024378,0.008591,-0.055893,-0.429502,-0.247272,-0.003797,-0.078569,1.000000,0.039638


## Label Encode categorical data

In [5]:
print(m_data_raw.shape)

(11027106, 14)


## Split data into Train-Test (Stratified 80/20)

In [6]:
train_data_new = pd.read_csv("train_data_new.csv")
test_data_new = pd.read_csv("test_data_new.csv")

In [7]:
# y = m_data_raw[["label"]] # labels

# X = m_data_raw.copy() # features
# X.drop("label", axis = 1, inplace = True)
# X.drop("sequence", axis = 1, inplace = True)
# X.drop("transcript_id", axis = 1, inplace = True)
# X.drop("gene_id", axis = 1, inplace = True)

# train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.20, random_state = 4266, stratify = y)

# train_data = train_y.join(train_X)
# test_data = test_y.join(test_X)

# train_X_og = train_X.copy()
# test_X_og = test_X.copy()

In [9]:
new_train_y = train_data_new[["label"]]
new_train_X = train_data_new.copy()
new_train_X = new_train_X.drop(["transcript_id", "transcript_position", "gene_id","label"], axis=1)

In [10]:
new_test_y = test_data_new[["label"]]
new_test_X = test_data_new.copy()
new_test_X = new_test_X.drop(["transcript_id", "transcript_position", "gene_id","label"], axis=1)

In [11]:
# print(train_X.shape)
# print(test_X.shape)

In [12]:
# X

## Scale data

In [13]:
## data normalisation - new_x = (x – min) / (max – min)
mm = MinMaxScaler()
# train_X = mm.fit_transform(train_X)
# test_X = mm.fit_transform(test_X)

In [14]:
new_train_X = mm.fit_transform(new_train_X)
new_test_X = mm.fit_transform(new_test_X)

# Logistic Regression Model

## Setup of helper function and dataframes to store results


a cross validation helper function below to remember the indexes that make up the best training set during cross validation.

In [15]:
def get_best(training_x, training_y, model):
  skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4266)
  lst_f1_stratified = []
  best_f1 = 0
  best_x_fold = []
  best_y_fold = []
  for train_index, val_index in skf.split(training_x, training_y):

      x_train_fold, x_val_fold = training_x[train_index], training_x[val_index]
      y_train_fold, y_val_fold = training_y.values.ravel()[train_index], training_y.values.ravel()[val_index]

      if len(best_x_fold) == 0:
        best_x_fold.extend(x_train_fold)
        best_y_fold.extend(y_train_fold)

      lr = model.fit(x_train_fold, y_train_fold)

      current_model_f1 = met.f1_score(y_true = y_val_fold, y_pred = model.predict(x_val_fold))
      lst_f1_stratified.append(current_model_f1)

      if current_model_f1 > best_f1:
        best_x_fold.clear()
        best_y_fold.clear()
        best_x_fold.extend(x_train_fold)
        best_y_fold.extend(y_train_fold)

  print('List of possible F1-score:', [round(x, 5) for x in lst_f1_stratified])
  print('\nMaximum F1-score That can be obtained from this model is:', round(max(lst_f1_stratified)*100, 5), '%')
  print('\nMinimum F1-score:', round(min(lst_f1_stratified)*100, 5), '%')
  print('\nAverage F1-score:', round(mean(lst_f1_stratified), 5))
  print('\nStandard Deviation is:', round(stdev(lst_f1_stratified), 5))

  return (best_x_fold, best_y_fold)

We define two variables to store model performance during training and testing.

In [16]:
## make predictions and compute metrics
# 1) we want to maximise f1 score - 2/(1/P + 1/R)
# 2) cfm.ravel() - tn, fp, fn, tp
# 3) class-specific accuracy for reference

column_template = {"model": [], "roc auc score": [],
                   "F1-score": [], "precision": [], "recall" : [],
                   "tn": [], "fp": [], "fn": [], "tp": [],
                   "class 0 accuracy": [], "class 1 accuracy ": []}

training_results = pd.DataFrame(column_template)
testing_results = pd.DataFrame(column_template)

Model is overly biased towards majority class, does not predict minority class at all. Model does not learn well, leading to poor testing performance subsequently.

## 2) Weighted Model

Make loss function weighted such that misclassifications of minority class examples have higher cost.

Use cross validation to get subset of training data with best F1-score on validation set.

In [17]:
# fit model
clf = LogisticRegression(random_state = 4266, max_iter = 500, class_weight = "balanced")

# CV
best_fold = get_best(new_train_X, new_train_y, clf)

List of possible F1-score: [0.67531, 0.67725, 0.67389, 0.67587, 0.677, 0.67668, 0.67807, 0.67507, 0.67632, 0.67504]

Maximum F1-score That can be obtained from this model is: 67.80682 %

Minimum F1-score: 67.38853 %

Average F1-score: 0.67605

Standard Deviation is: 0.00125


In [18]:
X_train = best_fold[0]
y_train = best_fold[1]
X_test = new_test_X
y_test = new_test_y

# fit model
clf = LogisticRegression(random_state = 4266,
                         max_iter = 500, class_weight = "balanced").fit(X_train, y_train)

cfm = met.confusion_matrix(y_true = y_train, y_pred = clf.predict(X_train))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
train_1 = {"model": "new_train_data",
  "roc auc score": met.roc_auc_score(y_true = y_train, y_score = clf.predict(X_train)),
	"F1-score": met.f1_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "precision" : met.precision_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "recall": met.recall_score(y_true = y_train, y_pred = clf.predict(X_train)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
training_results = training_results.append(train_1, ignore_index = True)

cfm = met.confusion_matrix(y_true = y_test, y_pred = clf.predict(X_test))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
test_1 = {"model": "new_train_data",
  "roc auc score": met.roc_auc_score(y_true = y_test, y_score = clf.predict(X_test)),
	"F1-score": met.f1_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "precision": met.precision_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "recall": met.recall_score(y_true = y_test, y_pred = clf.predict(X_test)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
testing_results = testing_results.append(test_1, ignore_index = True)

<ipython-input-18-224ee298bf57>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_results = training_results.append(train_1, ignore_index = True)
<ipython-input-18-224ee298bf57>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testing_results = testing_results.append(test_1, ignore_index = True)


In [19]:
training_results

,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,new_train_data,0.661698,0.676242,0.648875,0.706019,221899.0,137524.0,105823.0,254143.0,0.617376,0.706019


In [20]:
testing_results


,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,new_train_data,0.657501,0.154071,0.088008,0.617877,1468013.0,637797.0,38064.0,61548.0,0.697125,0.617877
